In [1]:
# print(model.model.transformer_module.decoder)
# print(model.class_predictor)
# - Get the linear layer with the class predictor 

import torch 
import random
from datasets import concatenate_datasets, load_dataset
from torch.utils.data import Dataset 
from datasets import load_dataset
import cv2
from torch.utils.data import DataLoader
from huggingface_hub import hf_hub_download
import json
from PIL import Image
import numpy as np
import os 
import torchvision as torchvision
from torch.utils.data import Dataset 
import torchvision.transforms as transforms
import torchvision as torchvision
import albumentations as A
import albumentations as A
from albumentations.pytorch import ToTensorV2

/home/chia/anaconda3/envs/bizon_py39/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch
print(torch.__version__)

2.0.1+cu117


In [3]:
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [4]:
torch.cuda.current_device()

0

## New Dataset

In [5]:
class NormalizeTransform(object):
    def __call__(self, img):
        img = img / 255.0
        return img
    
train_image_dir = '/media/data_16T/chloe/try/Dataset/urbanruraltrainimages'
train_mask_dir = '/media/data_16T/chloe/try/Dataset/urbanruraltrainmasks'
val_image_dir = '/media/data_16T/chloe/try/Dataset/urbanruralvalimages'
val_mask_dir = '/media/data_16T/chloe/try/Dataset/urbanruralvalmasks'

import torchvision.transforms as transforms

# train_transform expects an img. train_transform(img)
train_transform = transforms.Compose([
    transforms.RandomRotation(90),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.RandomHorizontalFlip(p=0.5),
    # transforms.Normalize(mean=[0.485, 0.456, 0.406],
    #                      std=[0.229, 0.224, 0.225])
    
])

image_transform = transforms.Compose([
    # transforms.RandomRotation(90),
    # transforms.RandomVerticalFlip(p=0.5),
    # transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
    NormalizeTransform()
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    # transforms.Normalize(mean=[0.485, 0.456, 0.406],
    #                      std=[0.229, 0.224, 0.225])

])

image_test_transform = transforms.Compose([
    NormalizeTransform()
])



In [6]:
repo_id = f"chloechia/loveda"
filename = "id2label.json"
id2label = json.load(open(hf_hub_download(repo_id, filename, repo_type="dataset"), "r"))
id2label = {int(k):v for k,v in id2label.items()}
print(id2label)

{0: 'Ignore', 1: 'Background', 2: 'Building', 3: 'Road', 4: 'Water', 5: 'Barren', 6: 'Forest', 7: 'Agricultural'}


In [7]:
from PIL import Image
import cv2 as cv2 
from transformers import Mask2FormerImageProcessor, Mask2FormerConfig, Mask2FormerForUniversalSegmentation, AutoImageProcessor

CONFIG = "facebook/mask2former-swin-large-cityscapes-semantic"
processor = Mask2FormerImageProcessor(ignore_index=0,reduce_labels=False, do_resize=False, do_rescale=False, do_normalize=False)
model = Mask2FormerForUniversalSegmentation.from_pretrained(CONFIG, id2label=id2label, ignore_mismatched_sizes=True)


Some weights of Mask2FormerForUniversalSegmentation were not initialized from the model checkpoint at facebook/mask2former-swin-large-cityscapes-semantic and are newly initialized because the shapes did not match:
- class_predictor.weight: found shape torch.Size([20, 256]) in the checkpoint and torch.Size([9, 256]) in the model instantiated
- class_predictor.bias: found shape torch.Size([20]) in the checkpoint and torch.Size([9]) in the model instantiated
- criterion.empty_weight: found shape torch.Size([20]) in the checkpoint and torch.Size([9]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
import os
import random
from PIL import Image
import torch
from torch.utils.data import Dataset

# class ApplyTransforms(object):
#     def __init__(self, transforms):
#         self.transforms = transforms

#     def __call__(self, image, mask):
#         # Apply the same random transformations to both image and mask
#         seed = torch.randint(0, 1000000, (1,)).item()
#         random_transforms = transforms.RandomApply(self.transforms, p=1.0)
#         transformed_image = random_transforms(image, seed)
#         transformed_mask = random_transforms(mask, seed)

#         return transformed_image, transformed_mask
        
class CustomSegmentationDataset(Dataset):
    def __init__(self, image_dir, mask_dir, transform=None, image_transform=None):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.transform = transform
        self.image_transform = image_transform
        self.image_files = os.listdir(image_dir)
        self.mask_files = os.listdir(mask_dir)
        self.num_files = len(self.image_files)

    def __len__(self):
        return self.num_files

    def __getitem__(self, index):
        image_path = os.path.join(self.image_dir, self.image_files[index])
        mask_path = os.path.join(self.mask_dir, self.mask_files[index])
        
        original_image = Image.open(image_path).convert('RGB')
        original_mask = Image.open(mask_path)
        
        if self.transform is not None:
            seed = np.random.randint(10000)
            random.seed(seed)
            image = self.transform(original_image)
            random.seed(seed)
            mask = self.transform(original_mask)
            # image = self.transform(original_image)
            # mask = self.transform(original_mask)
            mask = np.squeeze(mask)
            
        if self.image_transform is not None:
            image = self.image_transform(image)

        # return image, mask, original_image, original_mask
        return np.array(image), np.array(mask), np.array(original_image), np.array(original_mask)

def custom_collate_fn(batch):
    inputs = list(zip(*batch))
    images = inputs[0]
    segmentation_maps = inputs[1]
    
    batch = processor(
        images,
        segmentation_maps=segmentation_maps,
        return_tensors="pt",
    )

    batch["original_images"] = inputs[2]
    batch["original_segmentation_maps"] = inputs[3]
    return batch

In [9]:
train_dataset = CustomSegmentationDataset(train_image_dir, train_mask_dir, transform=train_transform, image_transform=image_transform)
val_dataset = CustomSegmentationDataset(val_image_dir, val_mask_dir, transform=test_transform, image_transform=image_test_transform)    

In [10]:
train_dataloader = DataLoader(train_dataset, batch_size=1, shuffle=True, collate_fn=custom_collate_fn)
val_dataloader = DataLoader(val_dataset, batch_size=1, shuffle=False, collate_fn=custom_collate_fn)

In [11]:
batch = next(iter(train_dataloader))
for k,v in batch.items():
  if isinstance(v, torch.Tensor):
    print(k,v.shape)
  else:
    print(k,v[0].shape)

pixel_values torch.Size([1, 3, 1024, 1024])
pixel_mask torch.Size([1, 1024, 1024])
mask_labels torch.Size([5, 1024, 1024])
class_labels torch.Size([5])
original_images (1024, 1024, 3)
original_segmentation_maps (1024, 1024)


In [12]:
model.class_predictor.weight.shape

torch.Size([9, 256])

In [13]:
# import torch.nn as nn 
# model.class_predictor.weight = nn.Parameter(model.class_predictor.weight[:8, :])
# model.class_predictor.bias = nn.Parameter(model.class_predictor.bias[:8])
# model.criterion.empty_weight = nn.Parameter(model.criterion.empty_weight[:8])

In [14]:
# model.criterion.empty_weight.shape

# Dataloading

# Image Segmentation Class

In [15]:
def color_palette():
    return [[0,0,0], [255, 255, 255], [255,0,0], [255,255,0], [0,0,255], [159,129,183], [0,255,0], [255,195, 128]]

palette = color_palette()
palette

[[0, 0, 0],
 [255, 255, 255],
 [255, 0, 0],
 [255, 255, 0],
 [0, 0, 255],
 [159, 129, 183],
 [0, 255, 0],
 [255, 195, 128]]

## Change the last Fully Connected Layer 

In [16]:
# from torch import nn 
# fc = nn.Linear(in_features=256, out_features=8)
# model.class_predictor = fc
# model.class_predictor

In [17]:
# import torch.nn as nn 
# model.class_predictor.weight = nn.Parameter(model.class_predictor.weight[:8, :])
# model.class_predictor.bias = nn.Parameter(model.class_predictor.bias[:8])
# model.criterion.empty_weight = nn.Parameter(model.criterion.empty_weight[:8])

In [18]:
# model.class_predictor

## Finetuning

In [19]:
import evaluate

metric = evaluate.load("mean_iou")

In [20]:
# Verify the labels
labels = [id2label[label] for label in batch["class_labels"][0].tolist()]
print(labels)

['Background', 'Road', 'Water', 'Forest', 'Agricultural']


In [21]:
# nn.init.xavier_uniform_(model.class_predictor.weight).shape


In [22]:
outputs = model(batch["pixel_values"],
                class_labels=batch["class_labels"],
                mask_labels=batch["mask_labels"])
outputs.loss

tensor([105.2779], grad_fn=<AddBackward0>)

In [23]:
import torch
from tqdm.auto import tqdm

device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=5e-5)

running_loss = 0.0
num_samples = 0
for epoch in range(5):
  print("Epoch:", epoch)
  model.train()
  for idx, batch in enumerate(tqdm(train_dataloader)): # this iterates through all the batches. one for loop = one batch is put through the training loop. train_dataloader is loading in the batches
      # Reset the parameter gradients
      optimizer.zero_grad()

      # Forward pass
      outputs = model(
          pixel_values=batch["pixel_values"].to(device),
          mask_labels=[labels.to(device) for labels in batch["mask_labels"]],
          class_labels=[labels.to(device) for labels in batch["class_labels"]],
      )

      # Backward propagation
      loss = outputs.loss
      loss.backward()

      batch_size = batch["pixel_values"].size(0)
      running_loss += loss.item()
      num_samples += batch_size

      if idx % 100 == 0:
        print("Loss:", running_loss/num_samples)

      # Optimization
      optimizer.step()

  model.eval()
  for idx, batch in enumerate(tqdm(val_dataloader)):
    if idx > 5:
      break

    pixel_values = batch["pixel_values"]
    
    # Forward pass
    with torch.no_grad():
      outputs = model(pixel_values=pixel_values.to(device))

    # get original images
    original_images = batch["original_images"]
    target_sizes = [(image.shape[0], image.shape[1]) for image in original_images]
    # predict segmentation maps
    predicted_segmentation_maps = processor.post_process_semantic_segmentation(outputs,
                                                                                  target_sizes=target_sizes)

    # get ground truth segmentation maps
    ground_truth_segmentation_maps = batch["original_segmentation_maps"]

    metric.add_batch(references=ground_truth_segmentation_maps, predictions=predicted_segmentation_maps)
  
  # NOTE this metric outputs a dict that also includes the mIoU per category as keys
  # so if you're interested, feel free to prcint them as well
  print("Mean IoU:", metric.compute(num_labels = len(id2label), ignore_index = 0)['mean_iou'])

# torch.save(model.state_dict(), 'model.pt')
# # with model.eval(), you're not updating the weights. 
# # {
#     'layer1.weight': tensor([...]),    # Example weight tensor of 'layer1'
#     'layer1.bias': tensor([...]),      # Example bias tensor of 'layer1'
#     'layer2.weight': tensor([...]),    # Example weight tensor of 'layer2'
#     'layer2.bias': tensor([...]),      # Example bias tensor of 'layer2'
#     ...
# }
# Each parameter in the model has a corresponding entry in the state dictionary. In the example above, 'layer1.weight' corresponds to the weight tensor of the first layer, 'layer1.bias' corresponds to the bias tensor of the first layer, and so on. The ellipsis ([...]) represents the values of the tensors.

# When you load the model.pt file using model.load_state_dict(torch.load('model.pt')), the state dictionary is loaded into the model, and the model's parameters are updated with the saved tensor values, restoring the model to the state it was in when it was saved.

Epoch: 0


  0%|          | 0/1366 [00:16<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 22.00 MiB (GPU 1; 10.76 GiB total capacity; 9.52 GiB already allocated; 6.31 MiB free; 9.81 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

# Color Palette


In [ ]:
from typing import Dict, Tuple

# building, road, barren, vegetation 
newidtocolor: Dict[int, Tuple[int, int, int]] = {
    "0": (70, 70, 70),
    "1": (128, 64, 128),
    "2": (152, 251, 152),
    "3":(107, 142, 35),
}

newidtocolor: Dict[int, Tuple[int, int, int]] = {
    int(key): value for key, value in newidtocolor.items()
}

newidtocolor

{0: (70, 70, 70), 1: (128, 64, 128), 2: (152, 251, 152), 3: (107, 142, 35)}